In [13]:
import pandas as pd
import numpy as np
import seaborn as sns

PATH_READ_BOUND = '123181_test_raw.csv'
PATH_READ_STOCK = '301085_test_raw.csv'

PATH_WRITE = 'test_log_ret_B+S.csv'

# df = pd.read_csv(PATH_READ, index_col='Timestamp')
stock = pd.read_csv(PATH_READ_STOCK, index_col='Timestamp')
bound = pd.read_csv(PATH_READ_BOUND, index_col='Timestamp')


Add Avg on Stock and Bound

In [14]:
bound['Avg'] = bound['Turnover']/bound['Volume']

nan_rows_Avg = bound[bound['Avg'].isnull()]

# 如遇涨/跌停，直接用高开低收的价格，因为都一样
for index, row in nan_rows_Avg.iterrows():
    if row['High'] == row['Low'] == row['Open'] == row['Close']:
        row['Avg'] = row['High']

bound.update(nan_rows_Avg)

#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

stock['Avg'] = stock['Turnover']/stock['Volume']

nan_rows_Avg = stock[stock['Avg'].isnull()]

# 如遇涨/跌停，直接用高开低收的价格，因为都一样
for index, row in nan_rows_Avg.iterrows():
    if row['High'] == row['Low'] == row['Open'] == row['Close']:
        row['Avg'] = row['High']

stock.update(nan_rows_Avg)

stock.to_csv('stock_temp.csv')

In [15]:
bound['BOUND_Log_Return_Avg_1min'] = np.log(bound['Avg'] / bound['Avg'].shift(1))

stock['STOCK_Log_Return_Avg_1min'] = np.log(stock['Avg'] / stock['Avg'].shift(1))

Add Target

In [16]:
bound['Target_Avg_1min'] = (bound['Avg'].shift(-1) - bound['Avg'])/bound['Avg']

print(bound.columns)

Index(['TradingDay', 'InstrumentID', 'PreClosePrice', 'Open', 'High', 'Low',
       'Volume', 'Turnover', 'Close', 'Avg', 'BOUND_Log_Return_Avg_1min',
       'Target_Avg_1min'],
      dtype='object')


New dataframe generated

In [17]:
temp1 = bound[['BOUND_Log_Return_Avg_1min','Target_Avg_1min']]
temp2 = stock[['STOCK_Log_Return_Avg_1min']]

df = pd.concat([temp1, temp2], axis=1)

df = df.dropna()

print(df)

df.to_csv(PATH_WRITE)

                     BOUND_Log_Return_Avg_1min  Target_Avg_1min  \
Timestamp                                                         
2023-07-07 09:31:00                  -0.006801        -0.011139   
2023-07-07 09:32:00                  -0.011201        -0.008134   
2023-07-07 09:33:00                  -0.008168        -0.002788   
2023-07-07 09:34:00                  -0.002792        -0.005101   
2023-07-07 09:35:00                  -0.005114        -0.002104   
...                                        ...              ...   
2023-07-20 14:53:00                  -0.000399         0.000785   
2023-07-20 14:54:00                   0.000785        -0.001522   
2023-07-20 14:55:00                  -0.001523         0.001276   
2023-07-20 14:56:00                   0.001275        -0.001110   
2023-07-20 14:57:00                  -0.001111        -0.000584   

                     STOCK_Log_Return_Avg_1min  
Timestamp                                       
2023-07-07 09:31:00           